# mnist 数据集下载

```sh
# 地址
http://yann.lecun.com/exdb/mnist/

# download
wget -c http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
wget -c http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz

wget -c http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz

wget -c http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


```

In [3]:
###################################################################################################
# tensorflow 对mnist数据集的封装
from tensorflow.examples.tutorials.mnist import input_data

# 从path中载入数据集
mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist",one_hot=True)

# training data size
print "Training data size: ", mnist.train.num_examples
print "Validating data size: ", mnist.validation.num_examples
print "Testing data size: ", mnist.test.num_examples
print "Example training data: ", mnist.train.images[0]
print "Example training data label: ", mnist.train.labels[0]


Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz
Training data size:  55000
Validating data size:  5000
Testing data size:  10000
Example training data:  [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.      

# 神经网络实例（全模型）

In [15]:
###################################################################################################
# 神经网络实例1
# tensorflow 使用mnist训练神经网络实例
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

###################################################################################################
# mnist 数据集相关常数
# 输入层节点数。 对于mnist数据集，这个等于图片的像素
INPUT_NODE = 784
# 输出层的节点数。这个等于类别的数目
OUTPUT_NODE = 10

###################################################################################################
# 配置神经网络参数
# 隐藏层节点数
LAYER1_NODE = 500
# 一个训练batch中的训练数据个数， batch越大，训练越接近梯度下降；batch越小，训练越接近随机梯度下降
BATCH_SIZE = 100
# 基础学习率
LEARNING_RATE_BASE = 0.8
# 学习率衰减率
LEARNING_RATE_DECAY = 0.99
# 描述模型复杂度的正则化项在损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练论数
TRAINING_STEPS = 30000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

###################################################################################################
# 辅助函数，给定了神经网络的输入和所有参数，计算神经网络前向传播结果
# 使用relu激活函数实现了去线性化， 实现了三层全连接神经网络
# 在函数中支持了传入计算参数平均值的类，可在测试时使用滑动平均模型
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class is None:
        # 计算隐藏层的前向传播结果，并且使用relu激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        # 计算输出层的前向传播结果
        # 输出层不加入激活函数原因： 在计算损失函数时会一并计算softmax函数
        # 不加入softmax不会影响预测结果原因：预测时使用的是不同类别对应节点输出值的相对大小，所以没有softmax层对最后分类结果
        # 的计算没有影响。于是在计算整个神经网络的前向传播时可以不加入最后的softmax层
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 首先通过avg_class.average函数计算得出变量的滑动平均值，然后再计算神经网络的前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)


###################################################################################################
# 训练模型
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
    
    # 生成隐藏层参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算当前参数下神经网络的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量
    # 此变量不需要计算滑动平均值，所以这里指定这个变量为不可训练的变量
    # 在tensorflow训练神经网络时，一般会将嗲表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0, trainable=False)
    
    # --------------------------------------------------------------------------------------------
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类；给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # 在所有代表神经网络参数的变量上使用滑动平均（辅助变量不需要）
    # tf.trainable_variables返回的就是图上集合GraphKeys.TRAINABLE_AVERAGES中的元素。这个集合的元素就是所有没有指定
    # trainable=False的参数
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # --------------------------------------------------------------------------------------------
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数
    # 当分类问题只有一个正确答案时，可以使用sparse_softmax_cross_entropy_with_logits函数来加速交叉熵计算
    # tf.argmax函数得到正确答案对应的类别编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失。 一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularization = regularizer(weights1) + regularizer(weights2)
    
    # 总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
    # --------------------------------------------------------------------------------------------
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,    # 基础学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减 
        global_step,           # 当前迭代的轮数
        mnist.train.num_examples/BATCH_SIZE,    # 总的迭代次数
        LEARNING_RATE_DECAY)                    # 学习率递减速度
    # 使用tf.train.GradientDescentOptimizer优化算法来优化损失函数
    # 这里的损失函数包括了交叉熵损失和L2正则化损失
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # --------------------------------------------------------------------------------------------
    # 在训练神经网络模型时，每过一遍数据既要通过反向传播算法来更新神经网络中的参数，又要更新每一个参数的滑动平均值
    # 这一过程，tensorflow提供了tf.control_dependecies和tf.group两种机制
    # train_op = tf.group(train_step, variable_averages_op)
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')
        
    # --------------------------------------------------------------------------------------------
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确
    # tf.equal 判断两个张量的每一维是否相等，相等返回True，否则返回False
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 将bool的数值转为实数型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # --------------------------------------------------------------------------------------------
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        # 准备验证数据。一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和判断训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        # 准备测试数据。在真实的应用中，这部分数据在训练时是不可见的，这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        # 迭代的训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果。
                # 由于mnist的数据集比较小，所以一次可以处理所有的验证数据。
                # 在神经网络比较复杂或者验证数据比较大时，可以将验证数据划分为更新的batch，以免发生内存溢出错误
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step, validation accuracy using average model is: %g" % (i, validate_acc))
            
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        # 在训练结束之后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print ("After %d training step, test accurary using average model is: %g" % (TRAINING_STEPS, test_acc))
        
###################################################################################################
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist", one_hot=True)
    train(mnist)

###################################################################################################
# tesorflow提供了一个主程序的入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()
        
    
    

Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step, validation accuracy using average model is: 0.059
After 1000 training step, validation accuracy using average model is: 0.9774
After 2000 training step, validation accuracy using average model is: 0.9824
After 3000 training step, validation accuracy using average model is: 0.9842
After 4000 training step, validation accuracy using average model is: 0.9858
After 5000 training step, validation accuracy using average model is: 0.9846
After 6000 training step, validation accuracy using average model is: 0.9848
After 7000 training step, validation accuracy using average model is: 0.985
After 8000 training step, validation accuracy using average model is: 0.9854
After 9000 training step, 

SystemExit: 

## 神经网络实例---不使用激活函数

In [17]:
###################################################################################################
# 神经网络实例---不适用激活函数
# tensorflow 使用mnist训练神经网络实例
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

###################################################################################################
# mnist 数据集相关常数
# 输入层节点数。 对于mnist数据集，这个等于图片的像素
INPUT_NODE = 784
# 输出层的节点数。这个等于类别的数目
OUTPUT_NODE = 10

###################################################################################################
# 配置神经网络参数
# 隐藏层节点数
LAYER1_NODE = 500
# 一个训练batch中的训练数据个数， batch越大，训练越接近梯度下降；batch越小，训练越接近随机梯度下降
BATCH_SIZE = 100
# 基础学习率
LEARNING_RATE_BASE = 0.8
# 学习率衰减率
LEARNING_RATE_DECAY = 0.99
# 描述模型复杂度的正则化项在损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练论数
TRAINING_STEPS = 5000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

###################################################################################################
# 辅助函数，给定了神经网络的输入和所有参数，计算神经网络前向传播结果
# 使用relu激活函数实现了去线性化， 实现了三层全连接神经网络
# 在函数中支持了传入计算参数平均值的类，可在测试时使用滑动平均模型
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class is None:
        # 计算隐藏层的前向传播结果，并且使用relu激活函数
        layer1 = tf.matmul(input_tensor, weights1) + biases1
        # 计算输出层的前向传播结果
        # 输出层不加入激活函数原因： 在计算损失函数时会一并计算softmax函数
        # 不加入softmax不会影响预测结果原因：预测时使用的是不同类别对应节点输出值的相对大小，所以没有softmax层对最后分类结果
        # 的计算没有影响。于是在计算整个神经网络的前向传播时可以不加入最后的softmax层
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 首先通过avg_class.average函数计算得出变量的滑动平均值，然后再计算神经网络的前向传播结果
        layer1 = tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1)
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)


###################################################################################################
# 训练模型
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
    
    # 生成隐藏层参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算当前参数下神经网络的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量
    # 此变量不需要计算滑动平均值，所以这里指定这个变量为不可训练的变量
    # 在tensorflow训练神经网络时，一般会将嗲表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0, trainable=False)
    
    # --------------------------------------------------------------------------------------------
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类；给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # 在所有代表神经网络参数的变量上使用滑动平均（辅助变量不需要）
    # tf.trainable_variables返回的就是图上集合GraphKeys.TRAINABLE_AVERAGES中的元素。这个集合的元素就是所有没有指定
    # trainable=False的参数
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # --------------------------------------------------------------------------------------------
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数
    # 当分类问题只有一个正确答案时，可以使用sparse_softmax_cross_entropy_with_logits函数来加速交叉熵计算
    # tf.argmax函数得到正确答案对应的类别编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失。 一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularization = regularizer(weights1) + regularizer(weights2)
    
    # 总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
    # --------------------------------------------------------------------------------------------
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,    # 基础学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减 
        global_step,           # 当前迭代的轮数
        mnist.train.num_examples/BATCH_SIZE,    # 总的迭代次数
        LEARNING_RATE_DECAY)                    # 学习率递减速度
    # 使用tf.train.GradientDescentOptimizer优化算法来优化损失函数
    # 这里的损失函数包括了交叉熵损失和L2正则化损失
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # --------------------------------------------------------------------------------------------
    # 在训练神经网络模型时，每过一遍数据既要通过反向传播算法来更新神经网络中的参数，又要更新每一个参数的滑动平均值
    # 这一过程，tensorflow提供了tf.control_dependecies和tf.group两种机制
    # train_op = tf.group(train_step, variable_averages_op)
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')
        
    # --------------------------------------------------------------------------------------------
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确
    # tf.equal 判断两个张量的每一维是否相等，相等返回True，否则返回False
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 将bool的数值转为实数型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # --------------------------------------------------------------------------------------------
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        # 准备验证数据。一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和判断训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        # 准备测试数据。在真实的应用中，这部分数据在训练时是不可见的，这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        # 迭代的训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果。
                # 由于mnist的数据集比较小，所以一次可以处理所有的验证数据。
                # 在神经网络比较复杂或者验证数据比较大时，可以将验证数据划分为更新的batch，以免发生内存溢出错误
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step, validation accuracy using average model is: %g" % (i, validate_acc))
            
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        # 在训练结束之后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print ("After %d training step, test accurary using average model is: %g" % (TRAINING_STEPS, test_acc))
        
###################################################################################################
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist", one_hot=True)
    train(mnist)

###################################################################################################
# tesorflow提供了一个主程序的入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()
        
    
    

Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step, validation accuracy using average model is: 0.0956
After 1000 training step, validation accuracy using average model is: 0.0958
After 2000 training step, validation accuracy using average model is: 0.0958
After 3000 training step, validation accuracy using average model is: 0.0958
After 4000 training step, validation accuracy using average model is: 0.0958
After 5000 training step, test accurary using average model is: 0.098


SystemExit: 

## 神经网络实例---不使用隐藏层

In [19]:
###################################################################################################
# 神经网络实例---不适用隐藏层
# tensorflow 使用mnist训练神经网络实例
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

###################################################################################################
# mnist 数据集相关常数
# 输入层节点数。 对于mnist数据集，这个等于图片的像素
INPUT_NODE = 784
# 输出层的节点数。这个等于类别的数目
OUTPUT_NODE = 10

###################################################################################################
# 配置神经网络参数
# 隐藏层节点数
LAYER1_NODE = 500
# 一个训练batch中的训练数据个数， batch越大，训练越接近梯度下降；batch越小，训练越接近随机梯度下降
BATCH_SIZE = 100
# 基础学习率
LEARNING_RATE_BASE = 0.8
# 学习率衰减率
LEARNING_RATE_DECAY = 0.99
# 描述模型复杂度的正则化项在损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练论数
TRAINING_STEPS = 5000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

###################################################################################################
# 辅助函数，给定了神经网络的输入和所有参数，计算神经网络前向传播结果
# 使用relu激活函数实现了去线性化， 实现了三层全连接神经网络
# 在函数中支持了传入计算参数平均值的类，可在测试时使用滑动平均模型
def inference(input_tensor, avg_class, weights1, biases1):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class is None:
        # 计算隐藏层的前向传播结果，并且使用relu激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        return layer1
    else:
        # 首先通过avg_class.average函数计算得出变量的滑动平均值，然后再计算神经网络的前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return layer1


###################################################################################################
# 训练模型
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
    
    # 生成隐藏层参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    # 计算当前参数下神经网络的前向传播结果
    y = inference(x, None, weights1, biases1)
    
    # 定义存储训练轮数的变量
    # 此变量不需要计算滑动平均值，所以这里指定这个变量为不可训练的变量
    # 在tensorflow训练神经网络时，一般会将嗲表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0, trainable=False)
    
    # --------------------------------------------------------------------------------------------
    # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类；给定训练轮数的变量可以加快训练早期变量的更新速度
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    # 在所有代表神经网络参数的变量上使用滑动平均（辅助变量不需要）
    # tf.trainable_variables返回的就是图上集合GraphKeys.TRAINABLE_AVERAGES中的元素。这个集合的元素就是所有没有指定
    # trainable=False的参数
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    # 计算使用了滑动平均之后的前向传播结果
    average_y = inference(x, variable_averages, weights1, biases1)
    
    # --------------------------------------------------------------------------------------------
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数
    # 当分类问题只有一个正确答案时，可以使用sparse_softmax_cross_entropy_with_logits函数来加速交叉熵计算
    # tf.argmax函数得到正确答案对应的类别编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失。 一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularization = regularizer(weights1)
    
    # 总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
    # --------------------------------------------------------------------------------------------
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,    # 基础学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减 
        global_step,           # 当前迭代的轮数
        mnist.train.num_examples/BATCH_SIZE,    # 总的迭代次数
        LEARNING_RATE_DECAY)                    # 学习率递减速度
    # 使用tf.train.GradientDescentOptimizer优化算法来优化损失函数
    # 这里的损失函数包括了交叉熵损失和L2正则化损失
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # --------------------------------------------------------------------------------------------
    # 在训练神经网络模型时，每过一遍数据既要通过反向传播算法来更新神经网络中的参数，又要更新每一个参数的滑动平均值
    # 这一过程，tensorflow提供了tf.control_dependecies和tf.group两种机制
    # train_op = tf.group(train_step, variable_averages_op)
    with tf.control_dependencies([train_step, variable_averages_op]):
        train_op = tf.no_op(name='train')
        
    # --------------------------------------------------------------------------------------------
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确
    # tf.equal 判断两个张量的每一维是否相等，相等返回True，否则返回False
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    # 将bool的数值转为实数型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # --------------------------------------------------------------------------------------------
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        # 准备验证数据。一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和判断训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        # 准备测试数据。在真实的应用中，这部分数据在训练时是不可见的，这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        # 迭代的训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果。
                # 由于mnist的数据集比较小，所以一次可以处理所有的验证数据。
                # 在神经网络比较复杂或者验证数据比较大时，可以将验证数据划分为更新的batch，以免发生内存溢出错误
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step, validation accuracy using average model is: %g" % (i, validate_acc))
            
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        # 在训练结束之后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print ("After %d training step, test accurary using average model is: %g" % (TRAINING_STEPS, test_acc))
        
###################################################################################################
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist", one_hot=True)
    train(mnist)

###################################################################################################
# tesorflow提供了一个主程序的入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()
        
    
    

Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step, validation accuracy using average model is: 0.0002
After 1000 training step, validation accuracy using average model is: 0.9234
After 2000 training step, validation accuracy using average model is: 0.9272
After 3000 training step, validation accuracy using average model is: 0.9262
After 4000 training step, validation accuracy using average model is: 0.928
After 5000 training step, test accurary using average model is: 0.9253


SystemExit: 

## 神经网络实例---不使用滑动平均

In [21]:
###################################################################################################
# 神经网络实例---不使用滑动平均
# tensorflow 使用mnist训练神经网络实例
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

###################################################################################################
# mnist 数据集相关常数
# 输入层节点数。 对于mnist数据集，这个等于图片的像素
INPUT_NODE = 784
# 输出层的节点数。这个等于类别的数目
OUTPUT_NODE = 10

###################################################################################################
# 配置神经网络参数
# 隐藏层节点数
LAYER1_NODE = 500
# 一个训练batch中的训练数据个数， batch越大，训练越接近梯度下降；batch越小，训练越接近随机梯度下降
BATCH_SIZE = 100
# 基础学习率
LEARNING_RATE_BASE = 0.8
# 学习率衰减率
LEARNING_RATE_DECAY = 0.99
# 描述模型复杂度的正则化项在损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练论数
TRAINING_STEPS = 5000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

###################################################################################################
# 辅助函数，给定了神经网络的输入和所有参数，计算神经网络前向传播结果
# 使用relu激活函数实现了去线性化， 实现了三层全连接神经网络
# 在函数中支持了传入计算参数平均值的类，可在测试时使用滑动平均模型
def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    # 当没有提供滑动平均类时，直接使用参数当前的取值
    if avg_class is None:
        # 计算隐藏层的前向传播结果，并且使用relu激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        # 计算输出层的前向传播结果
        # 输出层不加入激活函数原因： 在计算损失函数时会一并计算softmax函数
        # 不加入softmax不会影响预测结果原因：预测时使用的是不同类别对应节点输出值的相对大小，所以没有softmax层对最后分类结果
        # 的计算没有影响。于是在计算整个神经网络的前向传播时可以不加入最后的softmax层
        return tf.matmul(layer1, weights2) + biases2
    else:
        # 首先通过avg_class.average函数计算得出变量的滑动平均值，然后再计算神经网络的前向传播结果
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)


###################################################################################################
# 训练模型
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name="x-input")
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name="y-input")
    
    # 生成隐藏层参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    # 计算当前参数下神经网络的前向传播结果
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    # 定义存储训练轮数的变量
    # 此变量不需要计算滑动平均值，所以这里指定这个变量为不可训练的变量
    # 在tensorflow训练神经网络时，一般会将嗲表训练轮数的变量指定为不可训练的变量
    global_step = tf.Variable(0, trainable=False)
    
#     # --------------------------------------------------------------------------------------------
#     # 给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类；给定训练轮数的变量可以加快训练早期变量的更新速度
#     variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
#     # 在所有代表神经网络参数的变量上使用滑动平均（辅助变量不需要）
#     # tf.trainable_variables返回的就是图上集合GraphKeys.TRAINABLE_AVERAGES中的元素。这个集合的元素就是所有没有指定
#     # trainable=False的参数
#     variable_averages_op = variable_averages.apply(tf.trainable_variables())
#     # 计算使用了滑动平均之后的前向传播结果
#     average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    # --------------------------------------------------------------------------------------------
    # 计算交叉熵作为刻画预测值和真实值之间差距的损失函数
    # 当分类问题只有一个正确答案时，可以使用sparse_softmax_cross_entropy_with_logits函数来加速交叉熵计算
    # tf.argmax函数得到正确答案对应的类别编号
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(y, tf.argmax(y_, 1))
    # 计算在当前batch中所有样例的交叉熵平均值
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 计算L2正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失。 一般只计算神经网络边上权重的正则化损失，而不使用偏置项
    regularization = regularizer(weights1) + regularizer(weights2)
    
    # 总损失等于交叉熵损失和正则化损失之和
    loss = cross_entropy_mean + regularization
    
    # --------------------------------------------------------------------------------------------
    # 设置指数衰减的学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,    # 基础学习率，随着迭代的进行，更新变量时使用的学习率在这个基础上递减 
        global_step,           # 当前迭代的轮数
        mnist.train.num_examples/BATCH_SIZE,    # 总的迭代次数
        LEARNING_RATE_DECAY)                    # 学习率递减速度
    # 使用tf.train.GradientDescentOptimizer优化算法来优化损失函数
    # 这里的损失函数包括了交叉熵损失和L2正则化损失
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # --------------------------------------------------------------------------------------------
    # 在训练神经网络模型时，每过一遍数据既要通过反向传播算法来更新神经网络中的参数，又要更新每一个参数的滑动平均值
    # 这一过程，tensorflow提供了tf.control_dependecies和tf.group两种机制
    # train_op = tf.group(train_step, variable_averages_op)
#     with tf.control_dependencies([train_step, variable_averages_op]):
    with tf.control_dependencies([train_step]):
        train_op = tf.no_op(name='train')
        
    # --------------------------------------------------------------------------------------------
    # 检验使用了滑动平均模型的神经网络前向传播结果是否正确
    # tf.equal 判断两个张量的每一维是否相等，相等返回True，否则返回False
#     correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    # 将bool的数值转为实数型，然后计算平均值。这个平均值就是模型在这一组数据上的正确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    # --------------------------------------------------------------------------------------------
    # 初始化会话并开始训练过程
    with tf.Session() as sess:
        tf.initialize_all_variables().run()
        # 准备验证数据。一般在神经网络的训练过程中会通过验证数据来大致判断停止的条件和判断训练的效果
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        # 准备测试数据。在真实的应用中，这部分数据在训练时是不可见的，这个数据只是作为模型优劣的最后评价标准
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        # 迭代的训练神经网络
        for i in range(TRAINING_STEPS):
            # 每1000轮输出一次在验证数据集上的测试结果
            if i % 1000 == 0:
                # 计算滑动平均模型在验证数据上的结果。
                # 由于mnist的数据集比较小，所以一次可以处理所有的验证数据。
                # 在神经网络比较复杂或者验证数据比较大时，可以将验证数据划分为更新的batch，以免发生内存溢出错误
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                print("After %d training step, validation accuracy using average model is: %g" % (i, validate_acc))
            
            # 产生这一轮使用的一个batch的训练数据，并运行训练过程
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        # 在训练结束之后，在测试数据上检测神经网络模型的最终正确率
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print ("After %d training step, test accurary using average model is: %g" % (TRAINING_STEPS, test_acc))
        
###################################################################################################
# 主程序入口
def main(argv=None):
    mnist = input_data.read_data_sets("/Users/xxx/work5/tensorflow/mnist", one_hot=True)
    train(mnist)

###################################################################################################
# tesorflow提供了一个主程序的入口，tf.app.run会调用上面定义的main函数
if __name__ == '__main__':
    tf.app.run()
        
    
    

Extracting /Users/xxx/work5/tensorflow/mnist/train-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/xxx/work5/tensorflow/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step, validation accuracy using average model is: 0.107
After 1000 training step, validation accuracy using average model is: 0.9628
After 2000 training step, validation accuracy using average model is: 0.9736
After 3000 training step, validation accuracy using average model is: 0.979
After 4000 training step, validation accuracy using average model is: 0.9836
After 5000 training step, test accurary using average model is: 0.9814


SystemExit: 